In [1]:
from linreg_functions import *

In [2]:
data = pd.read_excel('grades.xlsx')

In [5]:
data

,Lectures,Homework,Grade
0,1.0,0.25,60
1,0.5,1.00,65
2,0.2,0.50,40
3,0.4,1.00,70
4,0.5,1.00,65
5,0.7,0.75,70
6,0.8,1.00,85
7,0.9,0.25,70
8,0.6,0.00,44
9,0.1,0.00,20


In [3]:
predictors = data.iloc[:,:-1]
y = data.iloc[:,-1].values.reshape(-1,1)

X = constructDesignMatrix(predictors)

In [4]:
B_hat, sigma2, se2 = Mult_LR_Least_squares(X,y)

print('Bhat: ', B_hat)
print('Biased estimator for sigma2: ', sigma2)
print('Unbiased estimator for sigma2(Se2): ', se2)



Bhat:  [[14.36757182]
 [46.19572347]
 [30.452053  ]]
Biased estimator for sigma2:  [[15.87198391]]
Unbiased estimator for sigma2(Se2):  19.83997988543445


In [6]:
TotalSS, RegSS, RSS = Mult_LR_partition_TSS (X,y)

print('Total Sum of Squares: ', TotalSS)
print('Regression Sum of Squares: ', RegSS)
print('Residual Sum of Squares: ', RSS)

Total Sum of Squares:  5781.600000000006
Regression Sum of Squares:  5543.520241374792
Residual Sum of Squares:  238.07975862521337


In [8]:
#testing simultaneous CIs

In [9]:
Mult_norm_LR_simul_CI(X,y,alpha=0.05)

Intervals of Bonferonni is  narrower:
Scheffe interval length for the first difference:  18.76712783345487
Bonferonni interval length for the first difference:  16.120132383338102


,lb,ub
0,6.307506,22.427638
1,36.578515,55.812932
2,21.925287,38.978819


In [13]:
C = np.array([0,1,0,0,0,1]).reshape(2,3)
C

array([[0, 1, 0],
       [0, 0, 1]])

In [14]:
#confidence intervals -> upper bound, lower bound
Mult_norm_LR_CR(X,y,C,alpha=0.05)

array([[55.84097376, 36.55047318],
       [39.00368093, 21.90042507]])

In [15]:
c0 = np.array([0,0]).reshape(-1,1)
c0

array([[0],
       [0]])

In [14]:
#returned False -> c0 is not in the confidence region

In [16]:
Mult_norm_LR_is_in_CR(X,y,C,c0,alpha=0.05)

(array([[5543.52024137]]), 154.1683030570119, False)

a way to test Mult_norm_LR_test_general:

With appropriate C and c0 values, I will check to see if Mult_norm_LR_test_general returns the same result
comparedd to testing single component of B. Thus, defining the function below:


In [17]:
def testingSingleComponentofb(X,Y,which_predictor,alpha):
    
    B_hat = computeBhat (X,Y)
    n = X.shape[0]
    k = X.shape[1] - 1
    
    B_hat_i =  B_hat[which_predictor]
    
    dof = n-k-1
    t_crit = t.ppf(1- (alpha/2),dof)
    
    se = np.sqrt(computeSe2 (X,Y))
    
    x_matrix = np.linalg.inv(np.dot(X.T,X))
    temp = np.sqrt(x_matrix[which_predictor,which_predictor])
    
    error_of_margin = t_crit * se * temp
    
    ub = B_hat_i + error_of_margin
    lb = B_hat_i - error_of_margin
    
    t_stat = (B_hat_i - 0) / (se * temp)
    
    return pd.DataFrame(np.array([lb,ub,t_stat,t_crit]).reshape(1,-1),
                         columns=['lb','ub','t_stat','t_crit'])
    

In [18]:
C_single = np.array([0,1,0]).reshape(1,-1)
c_single = np.array([0])

In [21]:
testingSingleComponentofb(X,y,which_predictor=1,alpha=0.1)

,lb,ub,t_stat,t_crit
0,[40.028859843483815],[52.36258709341766],[13.351043264309451],1.78229


In [23]:
Mult_norm_LR_test_general(X,y,C_single,c_single,alpha=0.1)

,lb,ub
0,40.02886,52.362587


Result is the same from both tests! confirming the correctness of Mult_norm_LR_test_general

In [24]:
#testing Mult_norm_LR_test_comp, thus declaring j:

In [7]:
j = np.array([1,2]).reshape(1,-1)

In [9]:
Mult_norm_LR_test_comp(X,y,j,alpha=0.1)

True

In [10]:
#there is a regression!
Mult_norm_LR_test_linear_reg(X,y,alpha=0.1)

(139.70579288350427, 2.806795605732417, 'Regression')

In [11]:
d = np.array([0,1,1,1,0,1]).reshape(2,3)

In [12]:
Mult_norm_LR_pred_CI (X,y,d,alpha=0.01,method='best')

Intervals of Bonferonni is  narrower:
Scheffe interval length for the first difference:  28.74903078741967
Bonferonni interval length for the first difference:  26.4842317731385


,lb,ub
0,27.028445,53.512677
1,-8.722971,70.430481


In [32]:
Mult_norm_LR_pred_CI (X,y,d,alpha=0.01,method='Scheffe')

Returning Scheffe Conf Intervals:


,lb,ub
0,25.896045,54.645076
1,-12.107375,73.814885


In [13]:
Mult_norm_LR_pred_CI (X,y,d,alpha=0.01,method='Bonferonni')

Returning Bonferonni Conf Intervals:


,lb,ub
0,27.028445,53.512677
1,-8.722971,70.430481
